 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dash_table

import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import plotly.express as px

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [3]:
username = 'postgres'
password = '123'

host = 'localhost'
database = 'airlines'
port = None

In [4]:
url = URL.create(
    "postgresql",
    username = 'postgres',
    password = 'qpa102188523',
    host = 'localhost',
    database = 'airlines')
    
engine = create_engine(url)

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [5]:
# con = connect(
#     host=host,
#     user=username,
#     password=password,
#     database=database
# )

# cursor = con.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [6]:
query = "SELECT * FROM reporting.top_reliability_roads;"
top_routes_df = pd.read_sql(query, engine)

top_routes_df

,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,cnt,delayed_ratio_reliability,year,nb
0,13930,Chicago O'Hare International,12953,LaGuardia,14219,0.41,2019,1
1,12892,Los Angeles International,12478,John F. Kennedy International,12797,0.39,2019,2
2,12892,Los Angeles International,12889,McCarran International,11628,0.38,2019,3
3,12889,McCarran International,12892,Los Angeles International,11616,0.37,2019,4
4,12892,Los Angeles International,14771,San Francisco International,14096,0.37,2019,4
5,12953,LaGuardia,13930,Chicago O'Hare International,14203,0.36,2019,5
6,14771,San Francisco International,12892,Los Angeles International,14088,0.34,2019,6
7,13830,Kahului Airport,12173,Hollywood-Burbank Midpoint,10718,0.27,2019,7
8,12478,John F. Kennedy International,12892,Los Angeles International,12736,0.27,2019,7
9,12173,Hollywood-Burbank Midpoint,13830,Kahului Airport,10725,0.25,2019,8


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [7]:
# w budowie schematu proszono o stworzenie widoku i rankingu według najwyższego odsetka opóźnień
top_routes_df['delayed_ratio_reliability'] = (top_routes_df['delayed_ratio_reliability'] * 100).astype(str) + '%'
top_routes_df

,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,cnt,delayed_ratio_reliability,year,nb
0,13930,Chicago O'Hare International,12953,LaGuardia,14219,41.0%,2019,1
1,12892,Los Angeles International,12478,John F. Kennedy International,12797,39.0%,2019,2
2,12892,Los Angeles International,12889,McCarran International,11628,38.0%,2019,3
3,12889,McCarran International,12892,Los Angeles International,11616,37.0%,2019,4
4,12892,Los Angeles International,14771,San Francisco International,14096,37.0%,2019,4
5,12953,LaGuardia,13930,Chicago O'Hare International,14203,36.0%,2019,5
6,14771,San Francisco International,12892,Los Angeles International,14088,34.0%,2019,6
7,13830,Kahului Airport,12173,Hollywood-Burbank Midpoint,10718,27.0%,2019,7
8,12478,John F. Kennedy International,12892,Los Angeles International,12736,27.0%,2019,7
9,12173,Hollywood-Burbank Midpoint,13830,Kahului Airport,10725,25.0%,2019,8


 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [8]:
top_routes_df = top_routes_df.loc[:, ['origin_airport_name', 'dest_airport_name', 'year', 'delayed_ratio_reliability', 'nb']].rename(columns={
    'origin_airport_name': 'Origin Airport',
    'dest_airport_name': 'Destination Airport',
    'year': 'Year',
    'delayed_ratio_reliability': 'Not reliable',
    'nb': 'Rank'
})
top_routes_df

,Origin Airport,Destination Airport,Year,Not reliable,Rank
0,Chicago O'Hare International,LaGuardia,2019,41.0%,1
1,Los Angeles International,John F. Kennedy International,2019,39.0%,2
2,Los Angeles International,McCarran International,2019,38.0%,3
3,McCarran International,Los Angeles International,2019,37.0%,4
4,Los Angeles International,San Francisco International,2019,37.0%,4
5,LaGuardia,Chicago O'Hare International,2019,36.0%,5
6,San Francisco International,Los Angeles International,2019,34.0%,6
7,Kahului Airport,Hollywood-Burbank Midpoint,2019,27.0%,7
8,John F. Kennedy International,Los Angeles International,2019,27.0%,7
9,Hollywood-Burbank Midpoint,Kahului Airport,2019,25.0%,8


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [9]:
query = "SELECT * FROM reporting.year_to_year_comparision;"
yoy_comparison_df = pd.read_sql(query, engine)
yoy_comparison_df['not_reliable_ratio'] = round(yoy_comparison_df['not_reliable_ratio'] * 100, 2)
yoy_comparison_df

,year,month,flights_amount,total,not_reliable_ratio
0,2019,1,30048,86826,35.0
1,2019,2,32069,79514,40.0
2,2019,3,32788,91789,36.0
3,2019,4,31434,89505,35.0
4,2019,5,36285,94256,38.0
5,2019,6,40590,92823,44.0
6,2019,7,38192,95971,40.0
7,2019,8,38370,98725,39.0
8,2019,9,27264,90789,30.0
9,2019,10,31612,95901,33.0


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [10]:
yoy_comparison_to_plot_df = yoy_comparison_df.pivot(index='month', columns='year', values='not_reliable_ratio')
yoy_comparison_to_plot_df

year,2019,2020
month,,
1,35.0,28.0
2,40.0,28.0
3,36.0,19.0
4,35.0,NaN
5,38.0,NaN
6,44.0,NaN
7,40.0,NaN
8,39.0,NaN
9,30.0,NaN


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [11]:
yoy_comparison_fig = px.bar(yoy_comparison_to_plot_df, x=yoy_comparison_to_plot_df.index, y=[2019, 2020],
             title="Percentage of delayed flights by month (2019 vs. 2020)",
             labels={"index": "Month", "value": "Delayed flights (%)", "variable": "Year"},
             barmode='group', # Set barmode to 'group' for grouped bars
             #color_discrete_map={'2019': '#14487A', '2020': '#FAA275'}
             )  

yoy_comparison_fig.update_layout(xaxis_title = 'Month')

month_label = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
yoy_comparison_fig.update_xaxes(tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ticktext=month_label)

yoy_comparison_fig.show()


 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [12]:
query = 'SELECT * FROM reporting.day_to_day_comparision'
day_to_day_comparison_df = pd.read_sql(query, engine)
day_to_day_comparison_df['not_reliable_ratio'] = (day_to_day_comparison_df['not_reliable_ratio']) * 100
day_to_day_comparison_df

,year,day_of_week,flights_amount,total,not_reliable_ratio
0,2019,1,61976,164325,38.0
1,2019,2,55064,162265,34.0
2,2019,3,55879,160092,35.0
3,2019,4,64782,162989,40.0
4,2019,5,64883,165283,39.0
5,2019,6,43244,127682,34.0
6,2019,7,56822,153106,37.0
7,2020,1,8692,37713,23.0
8,2020,2,7219,36396,20.0
9,2020,3,8150,37532,22.0


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [13]:
day_to_day_comparison_to_plot_df = day_to_day_comparison_df.pivot(index = 'day_of_week', columns = 'year', values = 'not_reliable_ratio')
day_to_day_comparison_to_plot_df

year,2019,2020
day_of_week,,
1,38.0,23.0
2,34.0,20.0
3,35.0,22.0
4,40.0,31.0
5,39.0,31.0
6,34.0,26.0
7,37.0,26.0


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [14]:
day_to_day_comparison_fig = px.bar(day_to_day_comparison_to_plot_df, x=day_to_day_comparison_to_plot_df.index, y=[2019,2020],
                            title="Percentage of delayed flights by day of week (2019 vs. 2020)",
                            labels={"index": "Day of week", "value": "Delayed flights (%)", "variable": "Year"},
                            barmode='group',
                            #color_discrete_map={'2019': '#14487A', '2020': '#FAA275'}
                            )  

day_to_day_comparison_fig.update_layout(xaxis_title = 'Day of week')

day_label = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
day_to_day_comparison_fig.update_xaxes(tickvals=[1, 2, 3, 4, 5, 6, 7], ticktext=day_label)

day_to_day_comparison_fig.show()

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [15]:
query = 'SELECT * FROM reporting.day_by_day_reliability'
day_by_day_reliability_df = pd.read_sql(query, engine)
day_by_day_reliability_df['not_reliable_ratio'] = (day_by_day_reliability_df['not_reliable_ratio']) * 100
day_by_day_reliability_df['year'] = pd.to_datetime(day_by_day_reliability_df['date_column']).dt.year
day_by_day_reliability_df = day_by_day_reliability_df.sort_values(by='date_column')
day_by_day_reliability_df

,date_column,not_reliable_ratio,year
145,2019-01-01,40.0,2019
74,2019-01-02,44.0,2019
193,2019-01-03,35.0,2019
319,2019-01-04,29.0,2019
181,2019-01-05,37.0,2019
...,...,...,...
453,2020-03-27,11.0,2020
449,2020-03-28,13.0,2020
455,2020-03-29,10.0,2020
451,2020-03-30,12.0,2020


 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [16]:
day_by_day_reliability_fig = px.line(day_by_day_reliability_df, x='date_column', y='not_reliable_ratio',
                                    color='year',
                                    title = 'Percentage of delayed flights in 2019 and 2020',
                                    labels={"date_column": "Date", "not_reliable_ratio": "Delayed flights (%)", "year": "Year"},
                                    #color_discrete_map={'2019': '#14487A', '2020': '#FAA275'}
                                    )

day_by_day_reliability_fig.update_xaxes(title_text="Date")
day_by_day_reliability_fig.update_yaxes(title_text="Delayed flights (%)")


day_by_day_reliability_fig.show()

 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [17]:
top_routes_table = dash_table.DataTable(id='datatable',
                                        columns=[{'name': col, 'id': col} for col in top_routes_df.columns],
                                        data=top_routes_df.to_dict('records'),)

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [18]:
top_routes_page_title = html.H3("TOP 10 reliability routes in 2019 and 2020",
                                style={'font-family': 'verdana', 'color': '#4444'}) # 'white'

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [19]:
top_routes_layout = html.Div([top_routes_page_title, top_routes_table])

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [20]:
yoy_comparison_component = dcc.Graph(id='yoy bar', figure=yoy_comparison_fig)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [21]:
day_to_day_comparison_component = dcc.Graph(id='dtd bar', figure=day_to_day_comparison_fig)

 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [22]:
comparison_layout = html.Div(yoy_comparison_component)

 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [23]:
day_by_day_layout = html.Div(day_to_day_comparison_component)

 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [31]:
app = dash.Dash()

app.layout = html.Div([
  dcc.Location(id='url', refresh=False),

  html.A(html.Button("Top report"), href="/"),
  html.A(html.Button("Comparison"), href="/comparison_reporting"),
  html.A(html.Button("Day by day reporting"), href="/day_by_day_reporting"),
  
  html.Div(id='page-content')
])

layout_page_one = html.Div(
    children=[top_routes_layout])

layout_page_two = html.Div(
  children=[comparison_layout])

layout_page_three = html.Div(
  children=[day_by_day_layout])


 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [32]:
@app.callback(
  Output('page-content', 'children'),
  Input('url', 'pathname')
)
def display_page_content(pathname):
 
  if pathname == "/comparison_reporting":
    return layout_page_two
  elif pathname == "/day_by_day_reporting":
    return layout_page_three
  else:
    return layout_page_one

 Tutaj uruchom aplikację

In [33]:
app.run()

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.